# Notebook Summary

**Project Overview**
- This exploration projects maps the elevation above sea-level (in feet) of each house in Broad Channel and their vulnerability to flooding.
- It will serve as a baseline for including potential data/animations in a current documentary I'm producing about Broad Channel, that addresses flooding, climate change and resiliency 
- The benchmark for flooding is the occurence of 'regular' tidal flooding by 2050, which corresponds to a mean higher-high water level (currently 6ft) + 21". More info on this presentation from NYC department of planning https://www.arcgis.com/apps/MapJournal/index.html?appid=7b39dec6cde84a07858bdf207b2123e1#
- We chose tidal flooding because tides have been the main cause of floods in Broad Channel, with less frequent (but sometimes more extreme) occurences from storms and rain
- The objective is to understand what % of homes are under threat from regular flooding, and to map these houses

**Elevation Data**
- Elevation  based on an NYC open data dataset provided by the department of building (DOB) in October 2022 
- We assume that most buildings / houses in the city haven't changed since 2022 so consider that the dataset is fresh enough 
- The DOB used a third party provider to create the dataset, through the use of LIDAR (Light Detection and Ranging) technology to create the dataset 
- More info on the dataset provided here: https://data.cityofnewyork.us/City-Government/Building-Elevation-and-Subgrade-BES-/bsin-59hv/about_data

# Import packages

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from sodapy import Socrata
import datetime as dt
import geopandas as gpd
import json
import shapely 
from shapely.geometry import shape
import os
import dotenv
from dotenv import find_dotenv, load_dotenv
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, MultiPolygon, Point
from shapely.wkt import loads
import numpy as np
import folium

In [2]:
# load env variables - we need our personal app_token to call the nyc open data API
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
app_token = os.getenv('APP_TOKEN')

In [3]:
#formatting for dataframe siplay, I like to see all columns in full
pd.set_option('display.max_colwidth', None)
##pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


# Create useful functions

In [4]:
# function to retrieve fulldataset from NYC open data platform API 
def retrieve_data_set(data_set, app_token, timeout = 10, where_clause = '0=0'):
# The Host Name for the API endpoint (the https:// part will be added automatically)
    data_url = 'data.cityofnewyork.us'
    
# Create the client to point to the API endpoint
    client = Socrata(data_url, app_token)  
    client.timeout = timeout

# Define parameters for pagination
    page_size = 10000  # Adjust this as needed
    offset = 0
    all_results = []
    
# normally it's best to avoid while loops but this works  fine
# this loops  queries the dataset 10000 entries at a time until there's no more data
    while True:
        # Query a chunk of data
        
        results = client.get(data_set, limit=page_size, offset=offset, where = where_clause)

        if not results:
            break  # No more data to retrieve

        all_results.extend(results)
        offset += page_size

    # Convert the list of dictionaries to a Pandas data frame
    df = pd.DataFrame.from_records(all_results)
    return df


The function above allows us to make api calls do the NYC open data platform datasets, and to overcome the 50k records per page limit. Arguments of the function are: 
- data_set = unique_code of dataset, provided by NYC open data
- timeout = request timeout limit, default to 10
- app_token = secret app_token, stored as an env variable
- where_clause = allows us to filter the data by inputting an SQL where clause to avoid querying entire dataset when you only need a subcategory (very useful for 311 calls dataset) 

In [5]:
# function that easily returns table of count with percentage
def show_unique_count(df_name, column_name, digits=2):
    
    count = df_name[column_name].value_counts(dropna=False)
    percentage = (df_name[column_name].value_counts \
                      (dropna=False, normalize=True) \
                      *100).round(digits)
    table = pd.concat([count,percentage],\
                    axis=1,\
                    keys=['counts', '%'])
    return(table)

In [6]:
# Distribution matrix
def distrib_matrix(df, col1, col2): 
    perc= (df.groupby([col1])[col2].\
    value_counts(normalize=True)*100).round(2).unstack([col2])
    
    
    values = df.groupby([col1])[col2].\
    value_counts().unstack([col2])
    
    table = pd.concat([perc,values],\
                    axis=1,\
                    keys=['%', 'counts'])
    return table

# Import DF  and preprocess


In [7]:
# elevation of all buildings in NYC as of 2022
building_elevation = retrieve_data_set('bsin-59hv', 'asE710T1weBMd3mkODQ79qdye', timeout = 100)

In [8]:
building_elevation.head()

,the_geom,bin,bbl,borough,block,lot,address,z_grade,z_floor,subgrade,notes1,x,y,latitude,longitude,pluto_bbl,council,borocd,ctlabel,boroct2020,nta2020,ntaname,cdta2020,cdtaname,notes2,notes3
0,"{'type': 'Point', 'coordinates': [-74.22274561187417, 40.52134422844183]}",5128004.0,5075340353,5,7534,353,78 SAVO LOOP,29.428,32.332,N,Property was Successfully Measured,922321.468334,129295.106289,40.52134423,-74.22274561,5075340353.0,51,503,226.01,5022601,SI0304,Annadale-Huguenot-Prince's Bay-Woodrow,SI03,SI03 South Shore (CD 3 Approximation),NaN,NaN
1,"{'type': 'Point', 'coordinates': [-74.24179250549321, 40.52875316810818]}",5155392.0,5075960125,5,7596,125,72 CHART LOOP,25.366,26.703,N,Property was Successfully Measured,917033.446429,132008.386296,40.52875317,-74.24179251,5075960125.0,51,503,226.02,5022602,SI0305,Tottenville-Charleston,SI03,SI03 South Shore (CD 3 Approximation),Attached Garage to Living Space,NaN
2,"{'type': 'Point', 'coordinates': [-74.24109012652094, 40.528883822921635]}",5148808.0,5075960131,5,7596,131,40 TIDES LANE,36.172,38.902,N,Property was Successfully Measured,917228.833178,132055.448538,40.52888382,-74.24109013,5075960131.0,51,503,226.02,5022602,SI0305,Tottenville-Charleston,SI03,SI03 South Shore (CD 3 Approximation),NaN,NaN
3,"{'type': 'Point', 'coordinates': [-74.24649640323727, 40.507045708438184]}",5088274.0,5079150042,5,7915,42,328 MAIN STREET,69.897,72.459,N,Property was Successfully Measured,915703.546202,124103.533995,40.50704571,-74.2464964,5079150042.0,51,503,244.01,5024401,SI0305,Tottenville-Charleston,SI03,SI03 South Shore (CD 3 Approximation),NaN,NaN
4,"{'type': 'Point', 'coordinates': [-74.24198032271697, 40.51005666426356]}",5087850.0,5078680123,5,7868,123,309 SLEIGHT AVENUE,74.907,79.122,N,Property was Successfully Measured,916962.418356,125196.988173,40.51005666,-74.24198032,5078680123.0,51,503,244.01,5024401,SI0305,Tottenville-Charleston,SI03,SI03 South Shore (CD 3 Approximation),NaN,NaN


In [48]:
# we want to transform this df into a geopandas df in order to use geometry field. 
# for this we need to geometry field to be transformed from a geojson fromat to wkt format
# however the .apply(wkt.loads) doesn't work because of a strange formatting issue (this works if you import the df as a csv instead of a df from the API)
# a github user provides a neat solution here https://gist.github.com/drmalex07/5a54fc4f1db06a66679e
# we apply each step through a lambda fucntion on the "the_geom" column

building_elevation['geometry'] = building_elevation['the_geom'].apply(lambda row: json.dumps(row))
building_elevation['geometry'] = building_elevation['geometry'].apply(lambda row: json.loads(row))
building_elevation['geometry'] = building_elevation['geometry'].apply(lambda row: shape(row))


In [10]:
## convert our df to a gdf
building_elevation_gdf = gpd.GeoDataFrame(building_elevation, geometry='geometry')

In [26]:
#convert the two elevation fields to float 
# z_floor = The elevation of what is estimated to be the lowest actively used floor.
# z_grade = The elevation of the building at it's lowest adjacent grade - the lowest point where the building touches the ground.
building_elevation_gdf['z_floor'] = building_elevation_gdf['z_floor'].astype(float)
building_elevation_gdf['z_grade'] = building_elevation_gdf['z_grade'].astype(float)


In [28]:
building_elevation_gdf.head()

,the_geom,bin,bbl,borough,block,lot,address,z_grade,z_floor,subgrade,notes1,x,y,latitude,longitude,pluto_bbl,council,borocd,ctlabel,boroct2020,nta2020,ntaname,cdta2020,cdtaname,notes2,notes3,geometry
0,"{'type': 'Point', 'coordinates': [-74.22274561187417, 40.52134422844183]}",5128004.0,5075340353,5,7534,353,78 SAVO LOOP,29.428,32.332,N,Property was Successfully Measured,922321.468334,129295.106289,40.52134423,-74.22274561,5075340353.0,51,503,226.01,5022601,SI0304,Annadale-Huguenot-Prince's Bay-Woodrow,SI03,SI03 South Shore (CD 3 Approximation),NaN,NaN,POINT (-74.22275 40.52134)
1,"{'type': 'Point', 'coordinates': [-74.24179250549321, 40.52875316810818]}",5155392.0,5075960125,5,7596,125,72 CHART LOOP,25.366,26.703,N,Property was Successfully Measured,917033.446429,132008.386296,40.52875317,-74.24179251,5075960125.0,51,503,226.02,5022602,SI0305,Tottenville-Charleston,SI03,SI03 South Shore (CD 3 Approximation),Attached Garage to Living Space,NaN,POINT (-74.24179 40.52875)
2,"{'type': 'Point', 'coordinates': [-74.24109012652094, 40.528883822921635]}",5148808.0,5075960131,5,7596,131,40 TIDES LANE,36.172,38.902,N,Property was Successfully Measured,917228.833178,132055.448538,40.52888382,-74.24109013,5075960131.0,51,503,226.02,5022602,SI0305,Tottenville-Charleston,SI03,SI03 South Shore (CD 3 Approximation),NaN,NaN,POINT (-74.24109 40.52888)
3,"{'type': 'Point', 'coordinates': [-74.24649640323727, 40.507045708438184]}",5088274.0,5079150042,5,7915,42,328 MAIN STREET,69.897,72.459,N,Property was Successfully Measured,915703.546202,124103.533995,40.50704571,-74.2464964,5079150042.0,51,503,244.01,5024401,SI0305,Tottenville-Charleston,SI03,SI03 South Shore (CD 3 Approximation),NaN,NaN,POINT (-74.24650 40.50705)
4,"{'type': 'Point', 'coordinates': [-74.24198032271697, 40.51005666426356]}",5087850.0,5078680123,5,7868,123,309 SLEIGHT AVENUE,74.907,79.122,N,Property was Successfully Measured,916962.418356,125196.988173,40.51005666,-74.24198032,5078680123.0,51,503,244.01,5024401,SI0305,Tottenville-Charleston,SI03,SI03 South Shore (CD 3 Approximation),NaN,NaN,POINT (-74.24198 40.51006)


The dataset is NYC-wide but we only want to map the elevation of houses in Broad Channel
**Below we hardcode a geopolygon that emcopasses all of Broad Channel**
Polygon was inferred from data in https://data.cityofnewyork.us/Environment/New-York-City-s-Flood-Vulnerability-Index/mrjc-v9pm/about_data



In [126]:
# hardcode the coordinates
coords = [
    (-73.834201, 40.596657),
    (-73.829483, 40.598134),
    (-73.828345, 40.597646),
    (-73.826048, 40.599461),
    (-73.8243, 40.598332),
    (-73.823793, 40.599851),
    (-73.82441, 40.600342),
    (-73.822945, 40.602583),
    (-73.822483, 40.605583),
    (-73.820032, 40.608924),
    (-73.819952, 40.611006),
    (-73.82086, 40.612265),
    (-73.820473, 40.610195),
    (-73.821631, 40.608496),
    (-73.8241, 40.61232),
    (-73.821658, 40.614909),
    (-73.820941, 40.615237),
    (-73.817571, 40.614163),
    (-73.816527, 40.614947),
    (-73.814161, 40.614649),
    (-73.816156, 40.612942),
    (-73.814771, 40.608851),
    (-73.815341, 40.606939),
    (-73.815708, 40.606439),
    (-73.81589, 40.606025),
    (-73.817254, 40.60394),
    (-73.818491, 40.59939),
    (-73.820712, 40.595171),
    (-73.832326, 40.594619),
    (-73.834847, 40.595543),
    (-73.834201, 40.596657)
]

# create multipolygon from coords 
multipolygon = MultiPolygon([Polygon(coords)])

#filter df
broad_channel_elevation = building_elevation_gdf[building_elevation_gdf.geometry.within(multipolygon)]

In [127]:
broad_channel_elevation.head(5)

,the_geom,bin,bbl,borough,block,lot,address,z_grade,z_floor,subgrade,notes1,x,y,latitude,longitude,pluto_bbl,council,borocd,ctlabel,boroct2020,nta2020,ntaname,cdta2020,cdtaname,notes2,notes3,geometry
434983,"{'type': 'Point', 'coordinates': [-73.81952698729849, 40.60876247508015]}",4296816.0,4153080062,4,15308,62,629 CROSS BAY BOULEVARD,6.174,8.867,N,Property was Successfully Measured,1034335.0,161128.0,40.6087939,-73.8196175,4153080062.0,32,414,1072.01,4107201,QN1403,Breezy Point-Belle Harbor-Rockaway Park-Broad Channel,QN14,QN14 The Rockaways (CD 14 Approximation),Attached Garage to Living Space,NaN,POINT (-73.81953 40.60876)
434987,"{'type': 'Point', 'coordinates': [-73.81970352090588, 40.60335171720414]}",4297541.0,4154770006,4,15477,6,12-30 CROSS BAY BOULEVARD,6.308,11.380,Y,Property was Successfully Measured,1034387.0,159141.0,40.6033397,-73.819445,4154770006.0,32,414,1072.01,4107201,QN1403,Breezy Point-Belle Harbor-Rockaway Park-Broad Channel,QN14,QN14 The Rockaways (CD 14 Approximation),NaN,Visible Door or Window in subgrade space,POINT (-73.81970 40.60335)
435033,"{'type': 'Point', 'coordinates': [-73.82017267143677, 40.59786219623899]}",4616339.0,4154860100,4,15486,100,20-57 CROSS BAY BOULEVARD,6.475,10.388,N,Property was Successfully Measured; No Address Plate or Identification on Building,1034147.0,156906.0,40.5972065,-73.8203258,4154860100,32,414,1072.01,4107201,QN1403,Breezy Point-Belle Harbor-Rockaway Park-Broad Channel,QN14,QN14 The Rockaways (CD 14 Approximation),"Commercial, Industrial or other Non-Residential Lowest Floor Active Use",NaN,POINT (-73.82017 40.59786)
435046,"{'type': 'Point', 'coordinates': [-73.81720159245235, 40.60749434282077]}",4297414.0,4154560025,4,15456,25,721 WALTON ROAD,6.445,10.729,N,Property was Successfully Measured; No Address Plate or Identification on Building,1034988.0,160654.0,40.6074892,-73.8172693,4154560025.0,32,414,1072.01,4107201,QN1403,Breezy Point-Belle Harbor-Rockaway Park-Broad Channel,QN14,QN14 The Rockaways (CD 14 Approximation),NaN,NaN,POINT (-73.81720 40.60749)
435060,"{'type': 'Point', 'coordinates': [-73.81688737933501, 40.60805663745413]}",4616729.0,4154570023,4,15457,23,203 NOEL ROAD,5.519,12.984,N,Property was Successfully Measured,1035092.0,160872.0,40.6080869,-73.8168931,4154570023.0,32,414,1072.01,4107201,QN1403,Breezy Point-Belle Harbor-Rockaway Park-Broad Channel,QN14,QN14 The Rockaways (CD 14 Approximation),NaN,NaN,POINT (-73.81689 40.60806)


In [128]:
print(len(broad_channel_elevation))

print(broad_channel_elevation.describe())


924
          z_grade     z_floor
count  924.000000  924.000000
mean     5.083393    9.541280
std      1.516511    3.813527
min      0.000000    0.000000
25%      4.574500    7.033000
50%      5.215500    8.750500
75%      5.921000   13.074750
max      9.969000   20.594000


**924 buildings seems reasonable for an island with a population of about 3000 people 
Median z_floor elevation is 8,8 feet**

# Analysis and Map 

### Determining flood risk per house

We now need to determine which houses are under threat from flooding based on their elevation
- We use the **z_floor** dimension because it corresponds to elevation of lowest floor in house 
- z_grade is the lowest point on the house, but it might not be habitable (could be a stilt for example)
- According to a presentation by NYC planning, in 2050, regular tidal flooding could happen in Broad Channel with a **Mean Higher-high Water Level (MHHW) + 21"** (medium range scenario) https://www.arcgis.com/apps/MapJournal/index.html?appid=7b39dec6cde84a07858bdf207b2123e1#
- Current MHHW is 6.01 feet https://tidesandcurrents.noaa.gov/datums.html?id=8517137
- 86.01 feet + 21 inches = **7.76 feet**

So a house that has a **z_floor <  7.76** is likely to experience regular flooding by 2050


In [130]:
## create categorical variable that determines whether house is flood prone
broad_channel_elevation['is_flood_prone'] = broad_channel_elevation['z_floor'].apply(lambda row: True if row <7.76 else False)


In [131]:
broad_channel_elevation.head(2)

,the_geom,bin,bbl,borough,block,lot,address,z_grade,z_floor,subgrade,notes1,x,y,latitude,longitude,pluto_bbl,council,borocd,ctlabel,boroct2020,nta2020,ntaname,cdta2020,cdtaname,notes2,notes3,geometry,is_flood_prone
434983,"{'type': 'Point', 'coordinates': [-73.81952698729849, 40.60876247508015]}",4296816.0,4153080062,4,15308,62,629 CROSS BAY BOULEVARD,6.174,8.867,N,Property was Successfully Measured,1034335.0,161128.0,40.6087939,-73.8196175,4153080062.0,32,414,1072.01,4107201,QN1403,Breezy Point-Belle Harbor-Rockaway Park-Broad Channel,QN14,QN14 The Rockaways (CD 14 Approximation),Attached Garage to Living Space,NaN,POINT (-73.81953 40.60876),False
434987,"{'type': 'Point', 'coordinates': [-73.81970352090588, 40.60335171720414]}",4297541.0,4154770006,4,15477,6,12-30 CROSS BAY BOULEVARD,6.308,11.380,Y,Property was Successfully Measured,1034387.0,159141.0,40.6033397,-73.819445,4154770006.0,32,414,1072.01,4107201,QN1403,Breezy Point-Belle Harbor-Rockaway Park-Broad Channel,QN14,QN14 The Rockaways (CD 14 Approximation),NaN,Visible Door or Window in subgrade space,POINT (-73.81970 40.60335),False


In [132]:
# Distribution of houses that are being flooded
show_unique_count(broad_channel_elevation,'is_flood_prone', digits=2)

,counts,%
False,596,64.5
True,328,35.5


**35.5% of buildings in Broad Channel will experience regular tidal flooding by 2050**

## Maps 
Let's build a basic map of building elevation in BC, down the line we can send this map to an animator to incorporate in the documentary

In [139]:
# Load your GeoDataFrame
gdf = broad_channel_elevation  # Replace with the actual path to your data file

# Initialize the Folium map at the centroid of the first point

m = folium.Map(location=[center.y, center.x], zoom_start=14.5, tiles="CartoDB Positron")

# Function to determine the color based on z_floor
def get_color(is_flood_prone):
    return '#BA0000' if is_flood_prone == True else '#15D6B6'

# Add points to the map using Marker
for _, row in gdf.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=0.7,  # Reduced radius for very small dot-like appearance
        popup=f"Address: {row['address']}, z_floor: {row['z_floor']} feet",
        color=get_color(row['is_flood_prone']),
        fill=True,
        fill_color=get_color(row['is_flood_prone'])
    ).add_to(m)

# Creating a custom legend
legend_html = '''
<div style="position: fixed; 
     top: 100px; right: 50px; width: 200px; height: 90px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     ">&nbsp;  <b>Legend</b> <br>
     &nbsp; Flood Prone &nbsp; <i class="fa fa-circle" style="color:#BA0000"></i><br>
     &nbsp; Not Flood Prone Prone &nbsp; <i class="fa fa-circle" style="color:#15D6B6"></i>
</div>
'''
## Create a title
title_html = '''
             <h3 align="center" style="font-size:16px"><b>2050 tidal flood risk per building in Broad Channel, NYC</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(legend_html))
# Display the map in the Jupyter Notebook
m

In [140]:
#export the map
m.save('broad_channel_flood_map.html')

## Conclusion + Next Steps


**Preliminary Conclusion** 
This first exploration gives a good idea of how urgent the flooding problem is in Broad Channel
By 2050, models show that 1/3 house in BC will be subject to regular tidal flooding

**Next Steps and Improvements**
- Add a 'block' dimension - We know from reporting that some blocks are more prone to flooding than others, according to residents - it's unclear whether this factor is already included in the z_floor value. Ideally, we want to add another level of granularity in the flood predictions by having individual projections of tidal risk per block, not Island wide 
- Incorporate time dimension, to plot 'how often' houses might get flooded. Not all high tides lead to street and house flooding 
- Add other scenarios (we use a medium range 2050 tidal flooding scenario, but we could use many more such as rain and storm related scenarios)

This is however a good first step, and a baseground for including animation and data in our documentary project